# Simple tool formatting with HF
Showcasing how to use the new template format from HF for tool calling

## Loading the models

In [ ]:
# from huggingface_hub import login
# login()

In [42]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.bfloat16,
        device_map="auto",
    )




Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [41]:
# del model
# import gc
# gc.collect()
# torch.cuda.empty_cache()

## Setting up the system messages and default tools 

In [29]:
dialogs = [
    [
        {"role": "system", "content": "You are a helpful chatbot"},
        {"role": "user", "content": "What is the weather today in San Francisco?"},
    ],
    [
        {"role": "system", "content": "You are a helpful chatbot"},
        {"role": "user", "content": "What is the weather today in San Francisco?"},
    ],
  ]

messages = [
        {"role": "system", "content": "You are a helpful chatbot"},
        {"role": "user", "content": "What is the weather today in San Francisco?"},
    ]

builtin_tools = ["code_interpreter", "wolfram_alpha", "brave_search"]

json_tools = [ 
    { "type": "function",
      "function": {
          "name": "spotify_trending_songs",
          "description": "Get top trending songs on Spotify",
          "parameters": {
            "n": {
              "param_type": "int",
              "description": "Number of trending songs to get",
              "required": "true"
            }
          }
        }
   },
    {
      "type": "function",
      "function": {
        "name": "get_current_temperature",
        "description": "Get the current temperature for a specific location",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state, e.g., San Francisco, CA"
            },
            "unit": {
              "type": "string",
              "enum": ["Celsius", "Fahrenheit"],
              "description": "The temperature unit to use. Infer this from the user's location."
            }
          },
          "required": ["location", "unit"]
        }
      }
    },
    {
      "type": "function",
      "function": {
        "name": "get_rain_probability",
        "description": "Get the probability of rain for a specific location",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state, e.g., San Francisco, CA"
            }
          },
          "required": ["location"]
        }
      }
    }
]


json_tools = [ 
    { 
      "tool_name": "spotify_trending_songs",
      "description": "Get top trending songs on Spotify",
      "parameters": {
        "n": {
          "param_type": "int",
          "description": "Number of trending songs to get",
          "required": "true"
        }
      }
    
   },
    {
      "type": "function",
      "function": {
        "name": "get_rain_probability",
        "description": "Get the probability of rain for a specific location",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state, e.g., San Francisco, CA"
            }
          },
          "required": ["location"]
        }
      }
    }
]



## Converting to input ids and checking how the prompt format was applied

In [21]:
# for messages in dialog:
# Shouldn't output the Environment instruction, but it is.
input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

print(tokenizer.decode(input_ids[0], skip_special_tokens=False))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Cutting Knowledge Date: December 2023
Today Date: 23 Jul 2024

You are a helpful chatbot<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the weather today in San Francisco?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [22]:
input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
        builtin_tools=builtin_tools
    ).to(model.device)

print(tokenizer.decode(input_ids[0], skip_special_tokens=False))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: wolfram_alpha, brave_search

Cutting Knowledge Date: December 2023
Today Date: 23 Jul 2024

You are a helpful chatbot<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the weather today in San Francisco?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [30]:
input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt",
            custom_tools=json_tools,
        ).to(model.device)
    
print(tokenizer.decode(input_ids[0], skip_special_tokens=False))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Cutting Knowledge Date: December 2023
Today Date: 23 Jul 2024

You are a helpful chatbot<|eot_id|><|start_header_id|>user<|end_header_id|>

Use the function'spotify_trending_songs' to 'Get top trending songs on Spotify':
{"name": "spotify_trending_songs", "description": "Get top trending songs on Spotify", "parameters": {
    "n": {
        "param_type": "int",
        "description": "Number of trending songs to get",
        "required": "true"
    }
}Use the function 'get_rain_probability' to 'Get the probability of rain for a specific location':
{"name": "get_rain_probability", "description": "Get the probability of rain for a specific location", "parameters": {
    "type": "object",
    "properties": {
        "location": {
            "type": "string",
            "description": "The city and state, e.g., San Francisco, CA"
        }
    },
    "required": [
        "location"
    ]
}

Think very care

In [43]:
input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt",
            custom_tools=json_tools,
            builtin_tools=builtin_tools
        ).to(model.device)
    
print(tokenizer.decode(input_ids[0], skip_special_tokens=False))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: wolfram_alpha, brave_search

Cutting Knowledge Date: December 2023
Today Date: 23 Jul 2024

You are a helpful chatbot<|eot_id|><|start_header_id|>user<|end_header_id|>

Use the function'spotify_trending_songs' to 'Get top trending songs on Spotify':
{"name": "spotify_trending_songs", "description": "Get top trending songs on Spotify", "parameters": {
    "n": {
        "param_type": "int",
        "description": "Number of trending songs to get",
        "required": "true"
    }
}Use the function 'get_rain_probability' to 'Get the probability of rain for a specific location':
{"name": "get_rain_probability", "description": "Get the probability of rain for a specific location", "parameters": {
    "type": "object",
    "properties": {
        "location": {
            "type": "string",
            "description": "The city and state, e.g., San Francisco, CA"
        }
    },
    "required": [
       

## Running inference 


In [32]:
attention_mask = torch.ones_like(input_ids)
outputs = model.generate(
    input_ids,
    max_new_tokens=400,
    eos_token_id=tokenizer.eos_token_id,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
    attention_mask=attention_mask,
)
response = outputs[0][input_ids.shape[-1]:]
print("\nOutput:\n")
print(tokenizer.decode(response, skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.



Output:

<function=get_rain_probability>{"location": "San Francisco, CA"}</function>
